<a href="https://colab.research.google.com/github/h708isgood/1091030/blob/1091031/1091114_gradientTape_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')
x_train=x_train.reshape(-1,784)
x_test=x_test.reshape(-1,784)

In [ ]:
batch_size = 64 #60000/64=#937.5 ===938
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)) 
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
train_dataset2 = train_dataset.shuffle(buffer_size=1024).batch(batch_size)
test_dataset2 = test_dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
# x_batch_train.shape (64,28,28) 每次64筆
for step, (x_batch_train, y_batch_train) in enumerate(train_dataset2):
    print(x_batch_train.shape)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
model = tf.keras.models.Sequential()                 #← 建立序列模型物件
model.add(layers.Dense(512, activation='relu', input_dim= 784)) #← 加入第一層
model.add(layers.Dense(10, activation='softmax')) 

adam=tf.keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,
         epsilon=1e-07,decay=0, amsgrad=False)             
model.compile(optimizer=adam,             #← 指定優化器
      loss=loss_fn,  #'sparse_categorical_crossentropy', #← 指定損失函數
      metrics=['acc'])
model2 = tf.keras.models.Sequential()                 #← 建立序列模型物件
model2.add(layers.Dense(512, activation='relu', input_dim= 784)) #← 加入第一層
model2.add(layers.Dense(10, activation='softmax')) 

adam=tf.keras.optimizers.Adam(lr=0.001,beta_1=0.9,beta_2=0.999,
         epsilon=1e-07,decay=0, amsgrad=False)             
model2.compile(optimizer=adam,             #← 指定優化器
      loss=loss_fn,  #'sparse_categorical_crossentropy', #← 指定損失函數
      metrics=['acc'])

In [ ]:
model2.fit(x_train, y_train, epochs=10, batch_size=64)


In [ ]:
model2.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.0785 - acc: 0.9794


[0.07851317524909973, 0.9793999791145325]

In [ ]:
def step_():
  for step, (x_batch_train, y_batch_train) in enumerate(train_dataset2):
    with tf.GradientTape() as tape:
      logits = model(x_batch_train)
      loss_value = loss_fn(y_batch_train, logits)

    #if step % 200 == 0:
    #    print('Training loss (for one batch) at step %s: %s'
    #         % (step, float(loss_value)))
    #    print('Seen so far: %s samples' % ((step + 1) * 64))
    grads = tape.gradient(loss_value, model.trainable_weights)
    adam.apply_gradients(zip(grads, model.trainable_weights))
    #train_acc_metric(y_batch_train, logits)
    train_acc_metric.update_state(y_batch_train, logits)
  
for epoch in range(10):
    step_()
    train_acc = train_acc_metric.result()
    print('Training acc over epoch: %s' % (float(train_acc),))
    train_acc_metric.reset_states()
    print(model.evaluate(x_test, y_test))


Training acc over epoch: 0.9484500288963318
313/313 [==============================] - 1s 2ms/step - loss: 0.1037 - acc: 0.9670
[0.10366448014974594, 0.9670000076293945]
Training acc over epoch: 0.9795166850090027
313/313 [==============================] - 1s 2ms/step - loss: 0.0796 - acc: 0.9760
[0.07958237826824188, 0.9760000109672546]
Training acc over epoch: 0.9874833226203918
313/313 [==============================] - 1s 2ms/step - loss: 0.0784 - acc: 0.9751
[0.07844088226556778, 0.9750999808311462]
Training acc over epoch: 0.9911666512489319
313/313 [==============================] - 1s 2ms/step - loss: 0.0663 - acc: 0.9807
[0.0662819966673851, 0.9807000160217285]
Training acc over epoch: 0.9943666458129883
313/313 [==============================] - 1s 2ms/step - loss: 0.0597 - acc: 0.9830
[0.059709496796131134, 0.9829999804496765]
Training acc over epoch: 0.9964166879653931
313/313 [==============================] - 1s 2ms/step - loss: 0.0664 - acc: 0.9812
[0.06644991785287857, 

In [ ]:
cc=0
for x_batch_val, y_batch_val in test_dataset :
  cc +=1

sum=0
for x_batch_val, y_batch_val in test_dataset2 :
    val_logits = model(x_batch_val)
    val_acc_metric.update_state(y_batch_val, val_logits)
    
    #sum +=val_acc
    #print('Validation acc: %s' % (float(val_acc),))
#print(sum/cc)
val_acc = val_acc_metric.result()
val_acc_metric.reset_states()
print(val_acc)
model.evaluate(x_test, y_test)


tf.Tensor(0.9802, shape=(), dtype=float32)
313/313 [==============================] - 1s 2ms/step - loss: 0.0755 - acc: 0.9802


[0.07551714777946472, 0.9801999926567078]